In [2]:
import sas7bdat
import pandas as pd
import numpy as np
import requests
import time

In [3]:
path = r"std_company_identifiers.sas7bdat"
iden= sas7bdat.SAS7BDAT(path,encoding='gb2312').to_data_frame()

[std_company_identifiers.sas7bdat] header length 122880 != 8192


In [4]:
iden.head()

,RepRisk_ID,name,headquarter_country,headquarter_country_code,sectors,url,all_ISINs,primary_ISIN,No_reported_risk_exposure
0,10,Acer Inc,Taiwan,TW,Technology Hardware and Equipment,https://www.acer.com,US0044341065 US0044342055 TW0002353000,TW0002353000,false
1,100,Rio Tinto PLC,United Kingdom of Great Britain and Northern I...,GB,Mining,https://www.riotinto.com/,BRRIOTBDR007 GB0007406639 GB0007188757 US76720...,GB0007188757,false
2,1000,Terrane Metals Corp,Canada,CA,Mining,terranemetals.com,CA88103A1167 US88103A3068 CA88103A1084 CA88103...,CA88103A1084,false
3,100000,BLUECOM Co Ltd,Korea; Republic of (South Korea),KR,Technology Hardware and Equipment,http://www.bluec.co.kr/eng/,KR7033560004,KR7033560004,false
4,10002,Daishin Securities Co Ltd,Korea; Republic of (South Korea),KR,Financial Services,http://www.daishin.co.kr,KR7003542008 KR7003541000 KR7003540002,KR7003540002,false


In [5]:
iden_clean=(
    iden[iden.headquarter_country_code=="US"].
    filter(items=['RepRisk_ID', 'name', "sectors","primary_ISIN", "No_reported_risk_exposure","all_ISINs"])
)

iden_clean.head()


,RepRisk_ID,name,sectors,primary_ISIN,No_reported_risk_exposure,all_ISINs
8,100077,Red Mountain Resources Inc,Oil and Gas,US75678V3024,false,US75678V1127 US75678V2034 US75678V3024
12,10010,PDC Energy Inc (formerly Petroleum Development...,Oil and Gas,US69327R1014,false,US69327RAK77 US69327RAF82 US69327RAD35 US69327...
14,10013,Old National Bancorp,Banks,US6800331075,false,US680033AA57 US680033AB31 US680033AC14 US68003...
15,100131,Citadel Exploration Inc,Oil and Gas | Financial Services,US17286B1044,false,US17286B1044
27,100280,RVPLUS Inc,Financial Services,US78348P1012,false,US78348P1012


In [6]:
path = r"std_issue_data.sas7bdat"
issue= sas7bdat.SAS7BDAT(path,encoding='gb2312').to_data_frame()
issue.head()

[std_issue_data.sas7bdat] header length 65536 != 8192


,RepRisk_ID,date,issue,high_severity,medium_severity,low_severity,high_reach_source,medium_reach_source,low_reach_source,news_count_for_issue
0,10,17225.0,26.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,10,17225.0,42.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,10,17439.0,26.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,10,17439.0,42.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,10,17500.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [7]:
issue_clean=(
    issue[issue.RepRisk_ID.isin(list(iden_clean.RepRisk_ID))].
    filter(items=["RepRisk_ID","issue","news_count_for_issue"]).
    groupby(["RepRisk_ID","issue"]).
    agg(np.sum).
    reset_index().
    pivot('RepRisk_ID', 'issue', 'news_count_for_issue').
    reset_index().
    fillna(0)
)
issue_clean.head()

issue,RepRisk_ID,4.0,6.0,11.0,13.0,16.0,18.0,19.0,21.0,23.0,...,42.0,44.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0
0,100077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10010,4.0,6.0,0.0,15.0,1.0,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100131,2.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
rep_issue=pd.merge(iden_clean, issue_clean, on='RepRisk_ID', how='left').fillna(0)
rep_issue["all_ISINs"]=rep_issue.all_ISINs.apply(lambda x: x.split(' '))
rep_issue.head()

,RepRisk_ID,name,sectors,primary_ISIN,No_reported_risk_exposure,all_ISINs,4.0,6.0,11.0,13.0,...,42.0,44.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0
0,100077,Red Mountain Resources Inc,Oil and Gas,US75678V3024,false,"[US75678V1127, US75678V2034, US75678V3024]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10010,PDC Energy Inc (formerly Petroleum Development...,Oil and Gas,US69327R1014,false,"[US69327RAK77, US69327RAF82, US69327RAD35, US6...",4.0,6.0,0.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10013,Old National Bancorp,Banks,US6800331075,false,"[US680033AA57, US680033AB31, US680033AC14, US6...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100131,Citadel Exploration Inc,Oil and Gas | Financial Services,US17286B1044,false,[US17286B1044],2.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100280,RVPLUS Inc,Financial Services,US78348P1012,false,[US78348P1012],0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [9]:
rep_issue["all_ISINs"]

0              [US75678V1127, US75678V2034, US75678V3024]
1       [US69327RAK77, US69327RAF82, US69327RAD35, US6...
2       [US680033AA57, US680033AB31, US680033AC14, US6...
3                                          [US17286B1044]
4                                          [US78348P1012]
                              ...                        
4130                         [US25457N5005, US25457N4016]
4131                                       [US1265241075]
4132                                       [US8428121094]
4133                         [US98976E4008, US98978K1079]
4134                                       [US76155D1046]
Name: all_ISINs, Length: 4135, dtype: object

In [10]:
ticker=pd.read_excel("esg_usstocks.xlsx")
ticker

,ticker
0,AAL
1,AAPL
2,ABNB
3,ABVC
4,ADM
...,...
215,XPEV
216,XPP
217,YANG
218,YINN


In [11]:
def get_isin(ticker):
    try:
        res = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey=771ddce3249eb58ee8b912fe5c57c826').json()
        return res[0]['isin']
    except:
        pass

In [12]:
isin=[]
for i in ticker.ticker:
    time.sleep(1)
    isin.append(get_isin(i))
isin

['US02376R1023',
 'US0378331005',
 'US0090661010',
 None,
 'US0394831020',
 'US12468P1049',
 'US02005N1000',
 'US02043Q1076',
 'US00165C1045',
 'US0079031078',
 'US0231351067',
 'US0374111054',
 'CA03765K1049',
 'US03842K2006',
 'US03969T1097',
 None,
 None,
 'US04962H5063',
 'US02156B1035',
 'US0463531089',
 'US0970231058',
 'US01609W1027',
 'US0605051046',
 'US0567521085',
 'US08975P1084',
 None,
 None,
 'US10316T1043',
 None,
 'US0997241064',
 'US09257W1009',
 'US08862E1091',
 'US1347481020',
 'PA1436583006',
 None,
 'US16954W1018',
 'US1630751048',
 'US12572Q1058',
 None,
 '',
 'US79466L3024',
 'US17275R1023',
 'US74913G8814',
 None,
 None,
 None,
 'US23804L1035',
 None,
 None,
 'US2561631068',
 None,
 None,
 None,
 None,
 'CA29250N1050',
 'US2948216088',
 None,
 None,
 'US29786A1060',
 'US28176E1082',
 None,
 'US3453708600',
 None,
 'US30303M1027',
 None,
 None,
 'US3364331070',
 None,
 'US3696041033',
 'US3755581036',
 None,
 'US37045V1008',
 None,
 'CA0679011084',
 'US02079K3059

In [13]:
isin_clear=list(filter(None, isin))
len(isin_clear)

127

In [14]:
pd_isin=pd.DataFrame(isin_clear,columns=["all_ISINs"])
pd_isin

,all_ISINs
0,US02376R1023
1,US0378331005
2,US0090661010
3,US0394831020
4,US12468P1049
...,...
122,US9311421039
123,US9831341071
124,US30231G1022
125,US98422D1054


In [15]:
demo=rep_issue[rep_issue.primary_ISIN.isin(isin)]
demo

,RepRisk_ID,name,sectors,primary_ISIN,No_reported_risk_exposure,all_ISINs,4.0,6.0,11.0,13.0,...,42.0,44.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0
10,10056,Nu Skin Enterprises Inc,Personal and Household Goods,US67018T1051,false,"[US67018T2042, US67018TAA34, US67018TAB17, US6...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
18,1010,Home Depot Inc; The,Retail,US4370761029,false,"[US437076CD22, US437076CB65, US437076CC49, US4...",9.0,15.0,1.0,3.0,...,18.0,5.0,3.0,5.0,2.0,6.0,6.0,3.0,3.0,1.0
19,1011,Advanced Micro Devices Inc (AMD),Technology Hardware and Equipment,US0079031078,false,"[US007903BD80, US007903AK33, US007903AL16, US0...",1.0,2.0,1.0,3.0,...,3.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,1.0
128,104619,Square Inc,Software and Computer Services | Financial Ser...,US8522341036,false,"[US852234AA18, US852234AB90, US852234AC73, US8...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
153,10608,Nuance Communications Inc,Software and Computer Services,US67020Y1001,false,"[US67020YAK64, US67020YAH36, US67020YAG52, US6...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318,112,Walmart Inc (formerly Wal-Mart Stores Inc; Wal...,Retail,US9311421039,false,"[US03027XAZ33, US03027XBA72, US03027XBB55, US0...",137.0,101.0,28.0,42.0,...,440.0,73.0,146.0,140.0,56.0,150.0,89.0,12.0,49.0,10.0
498,121027,Uber Technologies Inc (Uber),Software and Computer Services | Travel and Le...,US90353T1007,false,[US90353T1007],91.0,4.0,1.0,6.0,...,291.0,19.0,14.0,1.0,0.0,61.0,83.0,30.0,114.0,20.0
512,122001,State Street Bank and Trust Co,Financial Services,US78462F1030,false,"[US78462F1030, US85744NAB73, US857449AB82, US8...",0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
612,126148,American Airlines Group Inc,Airlines,US02376R1023,false,"[US02376RAD44, US02376RAC60, US02377BAC00, US0...",15.0,3.0,1.0,0.0,...,17.0,2.0,10.0,0.0,0.0,7.0,6.0,1.0,27.0,0.0
888,1403,3M Co (3M),General Industrials,US88579Y1010,false,"[US88579YBM21, US88579YBK64, US88579YBH36, US8...",74.0,56.0,1.0,79.0,...,5.0,11.0,4.0,2.0,0.0,5.0,20.0,0.0,6.0,1.0


In [16]:
def explode(df, col):
    df[col] = df[col].apply(lambda x: [x] if not isinstance(x, list) else x)
    return df.drop(col, axis=1).join(
        pd.DataFrame(list(df[col])).stack().reset_index(level=1, drop=True).rename(col)
    )

In [17]:
exp_rep_issue=explode(rep_issue,"all_ISINs")

In [18]:
demo2=pd.merge(pd_isin, exp_rep_issue, on='all_ISINs', how='left').dropna()


In [19]:
demo_clear=(
    demo2.
    reset_index(drop=True).
    drop(['RepRisk_ID', 'primary_ISIN'], axis=1)
)

In [20]:
a=["ISIN","name","sectors","No_reported_risk_exposure","Impacts_on_communities",
              "Impacts_on_landscapes_ecosystems_and_biodiversity ","Violation_of_international_standards",
              "Local_pollution","Climate_change_GHG_emissions_and_global_pollution",
              "Other_environmental_issues","Waste_issues","Violation_of_national_legislation",
              "Human_rights_abuses_and_corporate_complicity","Other_social_issues",
              "Products_in_health_and_environmental_issues","Supply_chain_issues","Other_issues",
              "Occupational_health_and_safety_issues","Local_participation_issues", "Controversial_products_and_services",
              "Corruption_bribery_extortion_and_money_laundering","Social_discrimination","Executive_compensation_issues",
              "Overuse_and_wasting_of_resources","Animal_mistreatment","Poor_employmen_conditions",
              "Misleading_communication","Freedom_of_association_and_collective_bargaining","Forced_labor",
              "Child_labor","Discrimination_in_employment","Fraud","Tax_evasion","Anti_competitive_practices","Tax_optimization"]
demo_clear.columns=a

In [21]:
outputpath='rep_demo.csv'
demo_clear.to_csv(outputpath,sep=',',index=False,header=True)

In [22]:
a=["ISIN","name","sectors","No_reported_risk_exposure","Impacts_on_communities",
              "Impacts_on_landscapes_ecosystems_and_biodiversity ","Violation_of_international_standards",
            "Local_pollution","Climate_change_GHG_emissions_and_global_pollution",
              "Other_environmental_issues","Waste_issues","Violation_of_national_legislation",
              "Human_rights_abuses_and_corporate_complicity","Other_social_issues",
              "Products_in_health_and_environmental_issues","Supply_chain_issues","Other_issues",
              "Occupational_health_and_safety_issues","Local_participation_issues", "Controversial_products_and_services",
              "Corruption_bribery_extortion_and_money_laundering","Social_discrimination","Executive_compensation_issues",
              "Overuse_and_wasting_of_resources","Animal_mistreatment","Poor_employmen_conditions",
              "Misleading_communication","Freedom_of_association_and_collective_bargaining","Forced_labor",
              "Child_labor","Discrimination_in_employment","Fraud","Tax_evasion","Anti_competitive_practices","Tax_optimization"]

In [23]:
len(a)

35

In [24]:
rep_issue

,RepRisk_ID,name,sectors,primary_ISIN,No_reported_risk_exposure,all_ISINs,4.0,6.0,11.0,13.0,...,42.0,44.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0
0,100077,Red Mountain Resources Inc,Oil and Gas,US75678V3024,false,"[US75678V1127, US75678V2034, US75678V3024]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10010,PDC Energy Inc (formerly Petroleum Development...,Oil and Gas,US69327R1014,false,"[US69327RAK77, US69327RAF82, US69327RAD35, US6...",4.0,6.0,0.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10013,Old National Bancorp,Banks,US6800331075,false,"[US680033AA57, US680033AB31, US680033AC14, US6...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100131,Citadel Exploration Inc,Oil and Gas | Financial Services,US17286B1044,false,[US17286B1044],2.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100280,RVPLUS Inc,Financial Services,US78348P1012,false,[US78348P1012],0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4130,99940,DirectView Holdings Inc,Technology Hardware and Equipment,US25457N4016,false,"[US25457N5005, US25457N4016]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4131,99946,CUBA Beverage Co,Technology Hardware and Equipment | Food and B...,US1265241075,false,[US1265241075],0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4132,99949,Southern Energy Co Inc,Utilities | Oil and Gas,US8428121094,false,[US8428121094],1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4133,99955,Zoom Technologies Inc (formerly Zoom Telephoni...,Technology Hardware and Equipment,US98976E4008,false,"[US98976E4008, US98978K1079]",0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [25]:
outputpath='rep.csv'
rep_issue.to_csv(outputpath,sep=',',index=False,header=True)